In [ ]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)

In [1]:
!nvidia-smi                             # GPU 정보 확인
!head /proc/cpuinfo                     # CPU 정보 확인
!/usr/local/cuda/bin/nvcc --version     # CUDA 정보 확인
!pkg-config --modversion opencv  

Tue Jun 22 06:59:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import cv2
import numpy as np
from matplotlib import pyplot as plt
import re
import os
from PIL import Image 
import math
from skimage.io import imread

# import threading
# import sys
# import json
from pathlib import Path
import io
import torch

In [ ]:
!pip install easyocr

In [4]:
import time
import easyocr

In [7]:
# EasyOCR - bus_num_fr
# blur 제외하니까 1% 감소
bus_num_fr_path = '/content/drive/Shareddrives/EyesOn/ocr test/bus_num_fr/'
file_list = os.listdir(bus_num_fr_path)
result_list = list()
reader = easyocr.Reader(['ug'], gpu = True)

start = time.time()
for file in file_list:
    folder_name = file.split(' ')[0]
    try:
        img = cv2.imread(os.path.join(bus_num_fr_path, file))
        result = reader.readtext(img)[0][-2]
        result = re.findall('\d+', result)
    except:
        result = 'None'

    if folder_name in result:
        result_list.append(1)
    else:
        result_list.append(0)

print(round(time.time() - start,2),' sec')
print(round(sum(result_list)/len(result_list)*100,2),' %')

2.09  sec
36.73  %


In [34]:
# EasyOCR - bus_num_fr / fastNlMeansDenoisingColored 추가
bus_num_fr_path = '/content/drive/Shareddrives/EyesOn/ocr test/bus_num_fr/'
file_list = os.listdir(bus_num_fr_path)
result_list = list()
reader = easyocr.Reader(['ug'], gpu = True)

start = time.time()
for file in file_list:
    folder_name = file.split(' ')[0]
    try:
        img = cv2.imread(os.path.join(bus_num_fr_path, file))
        dst = cv2.fastNlMeansDenoisingColored(img,None,4,4,7,21)
        result = reader.readtext(dst)[0][-2]
        result = re.findall('\d+', result)
    except:
        result = 'None'

    if folder_name in result:
        result_list.append(1)
    else:
        result_list.append(0)

print(round(time.time() - start,2),' sec')
print(round(sum(result_list)/len(result_list)*100,2),' %')

4.51  sec
39.8  %


## 테스트

In [145]:
# EasyOCR - bus_num_fr / BLUR 및 CLAHE(Contrast Limited Adaptive Histogram Equalization) 적용
# RPi camera 프레임별 이미지 추출한 데이터(98장)로 테스트
bus_num_fr_path = '/content/drive/Shareddrives/EyesOn/ocr test/bus_num_fr/'
file_list = os.listdir(bus_num_fr_path)
result_list = list()
reader = easyocr.Reader(['bn'], gpu = True)

start = time.time()
for file in file_list:
    folder_name = file.split(' ')[0]
    plate_num = ""
    try:
        img = cv2.imread(os.path.join(bus_num_fr_path, file))
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray_image, (3, 3), 2, 1)
        clahe2 = cv2.createCLAHE(clipLimit=4.5, tileGridSize=(10,1))
        cl2 = clahe2.apply(blurred)
        result = reader.readtext(cl2)[0][-2]
        plate_num = re.findall('\d+', result)
    except:
        plate_num = 'None'

    if folder_name in plate_num:
        result_list.append(1)
    else:
        result_list.append(0)

print(round(time.time() - start,2),' sec')
print(round(sum(result_list)/len(result_list)*100,2),' %')

2.08  sec
61.22  %


In [6]:
# EasyOCR - bus_num_fr / BLUR 및 CLAHE(Contrast Limited Adaptive Histogram Equalization) 적용
# 학습시킨 데이터에서 crop한 이미지 데이터(690장)로 테스트
bus_num_fr_path = '/content/drive/Shareddrives/EyesOn/yolov5/runs/hub/exp/crops/bus_num_fr/'
file_list = os.listdir(bus_num_fr_path)
result_list = list()
reader = easyocr.Reader(['bn'], gpu = True)

start = time.time()
for file in file_list:
    folder_name = file.split(' ')[0]
    plate_num = ""
    try:
        img = cv2.imread(os.path.join(bus_num_fr_path, file))
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray_image, (3, 3), 2, 1)
        clahe2 = cv2.createCLAHE(clipLimit=4.5, tileGridSize=(10,1))
        cl2 = clahe2.apply(blurred)
        result = reader.readtext(cl2)[0][-2]
        plate_num = re.findall('\d+', result)
    except:
        plate_num = 'None'

    if folder_name in plate_num:
        result_list.append(1)
    else:
        result_list.append(0)

print(round(time.time() - start,2),' sec')
print(round(sum(result_list)/len(result_list)*100,2),' %')

14.33  sec
78.59  %


## EasyOCR - bus_num_fr / BLUR 및 CLAHE(Contrast Limited Adaptive Histogram Equalization) 적용한 모델의 언어별 테스트

In [144]:
lang_codes = 'abq ady af ang ar as ava az be bg bh bho bn bs ch_sim ch_tra che cs cy da dar de en es et fa fr ga gom hi hr hu id inh is it ja kbd kn ko ku la lbe lez lt lv mah mai mi mn mr ms mt ne new nl no oc pl pt ro ru rs_cyrillic rs_latin sck sk sl sq sv sw ta tab te th tjk tl tr ug uk ur uz'
lang_codes = lang_codes.split(' ')

bus_num_fr_path = '/content/drive/Shareddrives/EyesOn/ocr test/bus_num_fr/'
file_list = os.listdir(bus_num_fr_path)
result_list = list()

best_exct_lang = ''
best_exct = 50
best_pctg_lang = ''
best_pctg = 1

for lang_code in lang_codes:
    result_list = list()
    reader = easyocr.Reader([lang_code])

    start = time.time()
    for file in file_list:
        folder_name = file.split(' ')[0]
        plate_num = ""
        try:
            img = cv2.imread(os.path.join(bus_num_fr_path, file))
            gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray_image, (3, 3), 2, 1)
            clahe2 = cv2.createCLAHE(clipLimit=4.5, tileGridSize=(10,1))
            cl2 = clahe2.apply(blurred)
            result = reader.readtext(cl2)[0][-2]
            plate_num = re.findall('\d+', result)
        except:
            plate_num = 'None'

        if folder_name in plate_num:
            result_list.append(1)
        else:
            result_list.append(0)

    execution_time = round(time.time() - start,2)           # 프로세스 실행 시간 계산
    pctg = round(sum(result_list)/len(result_list)*100,2)   # 퍼센트 계산

    if execution_time < best_exct:
        best_exct_lang = lang_code
        best_exct = execution_time
    if pctg > best_pctg:
        best_pctg_lang = lang_code
        best_pctg = pctg

    print('########## ', lang_code,' ##########')
    print(lang_code,' time : ', execution_time, ' sec')
    print(lang_code,' pctg : ', pctg,' % \n')

print('########## best execution time ##########')
print(best_exct_lang,' time : ', best_exct, ' sec \n')
print('########## best percentage ##########')
print(best_pctg_lang,' pctg : ', best_pctg,' %')

##########  abq  ##########
abq  time :  2.19  sec
abq  pctg :  54.08  % 

##########  ady  ##########
ady  time :  2.17  sec
ady  pctg :  54.08  % 



##########  af  ##########
af  time :  1.47  sec
af  pctg :  39.8  % 

##########  ang  ##########
ang  time :  2.11  sec
ang  pctg :  46.94  % 



##########  ar  ##########
ar  time :  2.16  sec
ar  pctg :  51.02  % 

##########  as  ##########
as  time :  2.1  sec
as  pctg :  61.22  % 

##########  ava  ##########
ava  time :  2.18  sec
ava  pctg :  54.08  % 

##########  az  ##########
az  time :  1.48  sec
az  pctg :  39.8  % 

##########  be  ##########
be  time :  2.18  sec
be  pctg :  54.08  % 

##########  bg  ##########
bg  time :  2.19  sec
bg  pctg :  54.08  % 

##########  bh  ##########
bh  time :  2.13  sec
bh  pctg :  46.94  % 

##########  bho  ##########
bho  time :  2.11  sec
bho  pctg :  46.94  % 

##########  bn  ##########
bn  time :  2.07  sec
bn  pctg :  61.22  % 



##########  bs  ##########
bs  time :  1.47  sec
bs  pctg :  39.8  % 



##########  ch_sim  ##########
ch_sim  time :  1.93  sec
ch_sim  pctg :  33.67  % 

##########  ch_tra  ##########
ch_tra  time :  2.74  sec
ch_tra  pctg :  45.92  % 

##########  che  ##########
che  time :  2.17  sec
che  pctg :  54.08  % 

##########  cs  ##########
cs  time :  1.49  sec
cs  pctg :  39.8  % 

##########  cy  ##########
cy  time :  1.49  sec
cy  pctg :  39.8  % 

##########  da  ##########
da  time :  1.47  sec
da  pctg :  39.8  % 

##########  dar  ##########
dar  time :  2.19  sec
dar  pctg :  54.08  % 

##########  de  ##########
de  time :  1.49  sec
de  pctg :  39.8  % 

##########  en  ##########
en  time :  1.47  sec
en  pctg :  24.49  % 

##########  es  ##########
es  time :  1.49  sec
es  pctg :  39.8  % 

##########  et  ##########
et  time :  1.51  sec
et  pctg :  39.8  % 

##########  fa  ##########
fa  time :  2.13  sec
fa  pctg :  51.02  % 

##########  fr  ##########
fr  time :  1.52  sec
fr  pctg :  39.8  % 

##########  ga  ##########
ga  time :  1.

##########  it  ##########
it  time :  1.53  sec
it  pctg :  39.8  % 

##########  ja  ##########
ja  time :  1.64  sec
ja  pctg :  20.41  % 



##########  kbd  ##########
kbd  time :  2.16  sec
kbd  pctg :  54.08  % 

##########  kn  ##########
kn  time :  1.46  sec
kn  pctg :  57.14  % 

##########  ko  ##########
ko  time :  1.58  sec
ko  pctg :  14.29  % 

##########  ku  ##########
ku  time :  1.47  sec
ku  pctg :  39.8  % 

##########  la  ##########
la  time :  1.48  sec
la  pctg :  39.8  % 

##########  lbe  ##########
lbe  time :  2.19  sec
lbe  pctg :  54.08  % 

##########  lez  ##########
lez  time :  2.19  sec
lez  pctg :  54.08  % 

##########  lt  ##########
lt  time :  1.47  sec
lt  pctg :  39.8  % 

##########  lv  ##########
lv  time :  1.47  sec
lv  pctg :  39.8  % 

##########  mah  ##########
mah  time :  2.11  sec
mah  pctg :  46.94  % 

##########  mai  ##########
mai  time :  2.13  sec
mai  pctg :  46.94  % 

##########  mi  ##########
mi  time :  1.5  sec
mi  pctg :  39.8  % 

##########  mn  ##########
mn  time :  2.22  sec
mn  pctg :  54.08  % 

##########  mr  ##########
mr  time :  2.13  sec
mr  pc

##########  sw  ##########
sw  time :  1.48  sec
sw  pctg :  39.8  % 

##########  ta  ##########
ta  time :  2.11  sec
ta  pctg :  57.14  % 



##########  tab  ##########
tab  time :  2.18  sec
tab  pctg :  54.08  % 



##########  te  ##########
te  time :  1.48  sec
te  pctg :  44.9  % 

##########  th  ##########
th  time :  2.15  sec
th  pctg :  39.8  % 

##########  tjk  ##########
tjk  time :  2.23  sec
tjk  pctg :  54.08  % 

##########  tl  ##########
tl  time :  1.47  sec
tl  pctg :  39.8  % 

##########  tr  ##########
tr  time :  1.47  sec
tr  pctg :  39.8  % 

##########  ug  ##########
ug  time :  2.13  sec
ug  pctg :  51.02  % 

##########  uk  ##########
uk  time :  2.17  sec
uk  pctg :  54.08  % 

##########  ur  ##########
ur  time :  2.13  sec
ur  pctg :  51.02  % 

##########  uz  ##########
uz  time :  1.5  sec
uz  pctg :  39.8  % 

########## best execution time ##########
hr  time :  1.46  sec 

########## best percentage ##########
as  pctg :  61.22  %


## 각도 조정 코드 추가한 모델 (별로임)

In [18]:
# EasyOCR - bus_num_fr
bus_num_fr_path = '/content/drive/Shareddrives/EyesOn/ocr test/bus_num_fr/'
file_list = os.listdir(bus_num_fr_path)
result_list = list()
reader = easyocr.Reader(['ug'], gpu = True)

start = time.time()
for file in file_list:
    folder_name = file.split(' ')[0]
    try:
        img = cv2.imread(os.path.join(bus_num_fr_path, file))
        # -------- 기울기 손대기 -------- 
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # 배경에 그레이 적용
        gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC) # 사이즈 정규화(키우기)
        blur = cv2.GaussianBlur(gray, (5,5), 0) # 이미지에 블러 처리
        canny = cv2.Canny(blur, 700, 350, apertureSize = 5, L2gradient = True) # 이미지 외곽선만 추출
        lines = cv2.HoughLinesP(canny, 1, np.pi / 180, 50, minLineLength = 3, maxLineGap = 150) # 직선 찾기

        angle = 0
        maxdim = 0
        # 각도 조정
        if not (lines is None):
            for i in lines:
                xdim = i[0][2] - i[0][0]
                ydim = i[0][3] - i[0][1]
                iangle = math.atan2(ydim, xdim)*180/np.pi
                dim = math.sqrt((xdim * xdim) + (ydim * ydim))
                if abs(angle) < 40 and maxdim < dim:
                    maxdim = dim
                    angle =iangle

        roih, roiw, roic = img.shape
        matrix = cv2.getRotationMatrix2D((roiw/2, roih/2), angle, 1)
        # -------- 기울기 끝 -------- 
        roi = cv2.warpAffine(img, matrix, (roiw, roih))
        roi = cv2.resize(roi, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
        dst = cv2.fastNlMeansDenoisingColored(roi,None,4,4,7,21)
        # blurred = cv2.GaussianBlur(roi, (3, 3), 0)

        result = reader.readtext(dst)[0][-2]
        result = re.findall('\d+', result)
    except:
        result = 'None'

    if folder_name in result:
        result_list.append(1)
    else:
        result_list.append(0)

print(round(time.time() - start,2),' sec')
print(round(sum(result_list)/len(result_list)*100,2),' %')

15.51  sec
37.76  %


In [15]:
# EasyOCR - license_plate
license_plate_path = '/content/drive/Shareddrives/EyesOn/ocr test/license_plate/'
file_list = os.listdir(license_plate_path)
result_list = list()
reader = easyocr.Reader(['ug'], gpu = True)

start = time.time()
for file in file_list:
    folder_name = file.split(' ')[0]
    try:
        img = cv2.imread(os.path.join(license_plate_path, file))
        # -------- 기울기 손대기 -------- 
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # 배경에 그레이 적용
        gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC) # 사이즈 정규화(키우기)
        blur = cv2.GaussianBlur(gray, (5,5), 0) # 이미지에 블러 처리
        canny = cv2.Canny(blur, 700, 350, apertureSize = 5, L2gradient = True) # 이미지 외곽선만 추출
        lines = cv2.HoughLinesP(canny, 1, np.pi / 180, 50, minLineLength = 3, maxLineGap = 150) # 직선 찾기

        angle = 0
        maxdim = 0
        # 각도 조정
        if not (lines is None):
            for i in lines:
                xdim = i[0][2] - i[0][0]
                ydim = i[0][3] - i[0][1]
                iangle = math.atan2(ydim, xdim)*180/np.pi
                dim = math.sqrt((xdim * xdim) + (ydim * ydim))
                if abs(angle) < 40 and maxdim < dim:
                    maxdim = dim
                    angle =iangle

        roih, roiw, roic = img.shape
        matrix = cv2.getRotationMatrix2D((roiw/2, roih/2), angle, 1)
        # -------- 기울기 끝 -------- 
        roi = cv2.warpAffine(img, matrix, (roiw, roih))
        roi = cv2.resize(roi, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
        # blurred = cv2.GaussianBlur(roi, (3, 3), 0)

        result = reader.readtext(roi)[0][-2]
        result = re.findall('\d+', result)
    except:
        result = 'None'

    if folder_name in result:
        result_list.append(1)
    else:
        result_list.append(0)

print(round(time.time() - start,2),' sec')
print(round(sum(result_list)/len(result_list)*100,2),' %')

5.99  sec
2.78  %


In [10]:
# EasyOCR - license_plate
license_plate_path = '/content/drive/Shareddrives/EyesOn/ocr test/license_plate/'
file_list = os.listdir(license_plate_path)
result_list = list()
reader = easyocr.Reader(['ug'], gpu = True)

start = time.time()
for file in file_list:
    folder_name = file.split(' ')[0]
    try:
        img = cv2.imread(os.path.join(license_plate_path, file))
        blurred = cv2.GaussianBlur(img, (3, 3), 0)
        result = reader.readtext(blurred)[0][-2]
        result = re.findall('\d+', result)
    except:
        result = 'None'

    if folder_name in result:
        result_list.append(1)
    else:
        result_list.append(0)

print(round(time.time() - start,2),' sec')
print(round(sum(result_list)/len(result_list)*100,2),' %')

4.35  sec
0.93  %
